In [1]:
from access_token import ACCESS_TOKEN
owner, repo = 'tensorflow', 'datasets'

Using perceval Github backend from the command line

In [2]:
!perceval git --json-line https://github.com/tensorflow/datasets > tf_analysis.json
!perceval github -t $ACCESS_TOKEN --json-line --sleep-for-rate --category issue $owner $repo >> tf_analysis.json
!perceval github -t $ACCESS_TOKEN --json-line --sleep-for-rate --category pull_request $owner $repo >> tf_analysis.json

[2019-03-24 07:27:48,462] - Sir Perceval is on his quest.
[2019-03-24 07:28:20,745] - Fetching commits: 'https://github.com/tensorflow/datasets' git repository from 1970-01-01 00:00:00+00:00 to 2100-01-01 00:00:00+00:00; all branches
[2019-03-24 07:28:24,825] - Fetch process completed: 696 commits fetched
[2019-03-24 07:28:24,826] - Sir Perceval completed his quest.
[2019-03-24 07:28:25,497] - Sir Perceval is on his quest.
[2019-03-24 07:28:30,880] - Getting info for https://api.github.com/users/piyush-kgp
[2019-03-24 07:28:32,340] - Getting info for https://api.github.com/users/rsepassi
[2019-03-24 07:28:33,856] - Getting info for https://api.github.com/users/tiaguinho
[2019-03-24 07:28:35,314] - Getting info for https://api.github.com/users/LoSealL
[2019-03-24 07:28:36,798] - Getting info for https://api.github.com/users/ksachdeva
[2019-03-24 07:28:38,313] - Getting info for https://api.github.com/users/brettkoonce
[2019-03-24 07:28:39,336] - Getting info for https://api.github.com/u

[2019-03-24 07:31:41,125] - Getting info for https://api.github.com/users/HanGuo97
[2019-03-24 07:31:43,317] - Getting info for https://api.github.com/users/Ecalpal
[2019-03-24 07:31:44,305] - Getting info for https://api.github.com/users/PulkitMishra
[2019-03-24 07:31:48,161] - Getting info for https://api.github.com/users/zh794390558
[2019-03-24 07:31:52,504] - Getting info for https://api.github.com/users/kaustumbh7
[2019-03-24 07:31:54,305] - Getting info for https://api.github.com/users/pyaf
[2019-03-24 07:31:56,474] - Getting info for https://api.github.com/users/joeyearsley
[2019-03-24 07:31:57,497] - Getting info for https://api.github.com/users/vinsalg
[2019-03-24 07:31:58,532] - Getting info for https://api.github.com/users/mtngld
[2019-03-24 07:31:59,645] - Getting info for https://api.github.com/users/VIGS25
[2019-03-24 07:32:00,660] - Getting info for https://api.github.com/users/AnthonyZhai
[2019-03-24 07:32:01,684] - Getting info for https://api.github.com/users/Kammerlo

[2019-03-24 07:37:16,341] - Getting info for https://api.github.com/users/drewszurko
[2019-03-24 07:37:17,396] - Getting info for https://api.github.com/users/cyfra
[2019-03-24 07:37:27,438] - Getting info for https://api.github.com/users/ayush1999
[2019-03-24 07:37:33,358] - Getting info for https://api.github.com/users/KumarArindam
[2019-03-24 07:37:39,937] - Getting info for https://api.github.com/users/ctiijima
[2019-03-24 07:37:42,500] - Getting info for https://api.github.com/users/ParthS007
[2019-03-24 07:37:50,269] - Getting info for https://api.github.com/users/captain-pool
[2019-03-24 07:37:53,697] - Getting info for https://api.github.com/users/aayu04
[2019-03-24 07:37:56,204] - Getting info for https://api.github.com/users/us
[2019-03-24 07:38:18,595] - Getting info for https://api.github.com/users/AS2910
[2019-03-24 07:38:24,125] - Getting info for https://api.github.com/users/ubershmekel
[2019-03-24 07:38:41,617] - Getting info for https://api.github.com/users/Shashankjai

In [22]:
import datetime
import json
from pprint import pprint

with open('tf_analysis.json') as f:
    line = f.readline()
    line = json.loads(line) #convert string to dict
time = datetime.datetime.utcfromtimestamp(line['timestamp']).strftime('%Y-%m-%d %H:%M:%S UTC %Z%z')
print ('The date and time of data retrieval : ',time)

The date and time of data retrieval :  2019-03-24 01:58:23 UTC 


In [103]:
import pandas as pd

class Code_Changes:
    """Class for Code_Changes for Git repositories.
    
    Objects are instantiated by specifying a file with the
    commits obtained by Perceval from a set of repositories.
        
    :param path: Path to file with one Perceval JSON document per line
    """

    @staticmethod
    def _summary(repo, cdata):
        """Compute a summary of a commit, suitable as a row in a dataframe"""
        
        summary = {
            'repo': repo,
            'hash': cdata['commit'],
            'author': cdata['Author'],
            'author_date': datetime.datetime.strptime(cdata['AuthorDate'],
                                                      "%a %b %d %H:%M:%S %Y %z"),
            'commit': cdata['Commit'],
            'commit_date': datetime.datetime.strptime(cdata['CommitDate'],
                                                      "%a %b %d %H:%M:%S %Y %z"),
            'files_no': len(cdata['files'])
        }
        actions = 0
        for file in cdata['files']:
            if 'action' in file:
                actions += 1
        summary['files_action'] = actions
        if 'Merge' in cdata:
            summary['merge'] = True
        else:
            summary['merge'] = False
        return summary;
    
    def __init__(self, path):
        """Initilizes self.df, the dataframe with one row per commit.
        """

        self.df = pd.DataFrame(columns=['hash', 'author', 'author_date',
                                        'commit', 'commit_date',
                                        'files_no', 'files_action',
                                        'merge','repo'])
        commits = []
        with open(path) as commits_file:
            for line in commits_file:
                line = json.loads(line)
                if(line['category']=="commit"):
                    commit = line
                    commits.append(self._summary(repo=commit['origin'],cdata=commit['data']))
        
        self.df = self.df.append(commits, sort=False)
        self.df['author_date'] = pd.to_datetime(self.df['author_date'], utc=True)
        self.df['commit_date'] = pd.to_datetime(self.df['commit_date'], utc=True)
        
    def total_count(self):
        
        return len(self.df.index)
    
    def count(self, since = None, until = None, empty=True, merge=True, date='author_date'):
        """Count number of commits
        
        :param since: Period start
        :param until: Period end
        :param empty: Include empty commits
        :param merge: Include merge commits
        :param  date: Kind of date ('author_date' or 'commit_date')
        """
        
        df = self.df
        if since:
            df = df[df[date] >= since]
        if until:
            df = df[df[date] < until]
        if not empty:
            df = df[df['files_action'] != 0]
        if not merge:
            df = df[df['merge'] == False]
        return df['hash'].nunique()
    
    def by_month(self):
        
        return self.df['author_date'] \
            .groupby([self.df.author_date.dt.year.rename('year'),
                      self.df.author_date.dt.month.rename('month')]) \
            .agg('count')
            
    def unique_users(self):
        return self.df.author.nunique()

In [104]:
changes = Code_Changes('tf_analysis.json')
print("Code changes total count:", changes.total_count())
print("Code changes count all period:", changes.count())
print("Code changes count from 2018-09-12 to 2019-03-10:",
      changes.count(since="2018-09-12", until="2019-03-10"))
print("Code changes count from 2018-09-12 to 2019-03-10:(no merge commits):",
      changes.count(since="2018-09-12", until="2019-03-10", merge=False))
print("Code changes count from 2018-09-12 to 2019-03-10:(no empty commits):",
      changes.count(since="2018-09-12", until="2019-03-10", empty=False))

Code changes total count: 696
Code changes count all period: 696
Code changes count from 2018-09-12 to 2019-03-10: 617
Code changes count from 2018-09-12 to 2019-03-10:(no merge commits): 600
Code changes count from 2018-09-12 to 2019-03-10:(no empty commits): 614


In [137]:
changes.df.head()

,hash,author,author_date,commit,commit_date,files_no,files_action,merge,repo
0,680c6bda1b7d6db9d74f9c1373ebe737938926a7,Ryan Sepassi <rsepassi@google.com>,2018-09-12 21:23:17+00:00,Ryan Sepassi <rsepassi@google.com>,2018-09-12 21:23:17+00:00,1,1,False,https://github.com/tensorflow/datasets
1,87fd7f9fc4e36aa917726f3e8f89b7c89c99fb66,Ryan Sepassi <rsepassi@google.com>,2018-09-11 19:10:03+00:00,Ryan Sepassi <rsepassi@google.com>,2018-09-12 21:28:48+00:00,11,11,False,https://github.com/tensorflow/datasets
2,fb0f20383bb2a83477770517767b53ad97ec0840,Ryan Sepassi <rsepassi@google.com>,2018-09-12 20:12:22+00:00,Ryan Sepassi <rsepassi@google.com>,2018-09-12 21:28:57+00:00,2,2,False,https://github.com/tensorflow/datasets
3,ca9ffcccb2ae3409c4210475b6407b871ad51ba9,Ryan Sepassi <rsepassi@google.com>,2018-09-13 19:53:54+00:00,Ryan Sepassi <rsepassi@google.com>,2018-09-13 19:59:39+00:00,5,5,False,https://github.com/tensorflow/datasets
4,3f650ac05ea4e5d1eeef55a9d3a70a75b1cb5843,Dustin Tran <trandustin@google.com>,2018-09-20 18:59:48+00:00,Copybara-Service <copybara-piper@google.com>,2018-09-21 23:28:12+00:00,2,2,False,https://github.com/tensorflow/datasets


In [105]:
changes.by_month()

year  month
2018  9         10
      10        28
      11       107
      12       137
2019  1        122
      2        148
      3        144
Name: author_date, dtype: int64

In [107]:
print('Number of different identities : ',changes.unique_users()) 

Number of different identities :  42


In [51]:
# Create a dict having all commits
commits = {}
with open('tf_analysis.json') as commits_file:
    for line in commits_file:
        line = json.loads(line)
        if(line['category']=="commit"):
            commit = line
            commits[commit['data']['commit']] = commit
            
# Find commits in master branch.
# Start by adding head to an empty todo list. Then loop until todo set is empty:
# for each commit in the todo list, add it to the master set, and go backwards
# (finding parents), adding them to the todo set.
todo = set()
for id, commit in commits.items():
    if 'HEAD -> refs/heads/master' in commit['data']['refs']:
        todo.add(id)

        
master = set()
while len(todo) > 0:
    current = todo.pop()
    master.add(current)
    for parent in commits[current]['data']['parents']:
        if parent not in master:
            todo.add(parent)
    
code_commits = len(master)
    
print("Code Commits (master branch):", code_commits)

Code Commits (master branch): 677


In [136]:
with open('./tf_analysis.json') as pr_file:
            for line in pr_file:
                line = json.loads(line)
                if(line['category']=="pull_request"):
                    pr = line
                    break
pprint(pr)

{'backend_name': 'GitHub',
 'backend_version': '0.19.0',
 'category': 'pull_request',
 'data': {'_links': {'comments': {'href': 'https://api.github.com/repos/tensorflow/datasets/issues/3/comments'},
                     'commits': {'href': 'https://api.github.com/repos/tensorflow/datasets/pulls/3/commits'},
                     'html': {'href': 'https://github.com/tensorflow/datasets/pull/3'},
                     'issue': {'href': 'https://api.github.com/repos/tensorflow/datasets/issues/3'},
                     'review_comment': {'href': 'https://api.github.com/repos/tensorflow/datasets/pulls/comments{/number}'},
                     'review_comments': {'href': 'https://api.github.com/repos/tensorflow/datasets/pulls/3/comments'},
                     'self': {'href': 'https://api.github.com/repos/tensorflow/datasets/pulls/3'},
                     'statuses': {'href': 'https://api.github.com/repos/tensorflow/datasets/statuses/40be02c39c6b52d6427710012720f7150cadeadb'}},
          'ad

In [132]:
class pr_statistics:
    
    @staticmethod
    def _summary(pr_data):
        """Compute a summary of a pull_request, suitable as a row in a dataframe"""
        
        summary = {
            'base_repo': pr_data['base']['label'],
            'title': pr_data['title'],
            'state': pr_data['state'],
            'commits':pr_data['commits'],
            'commits_data': pr_data['commits_data'],
            'comments':pr_data['comments'],
            'changed_files': pr_data['changed_files'],
            'additions': pr_data['additions'],
            'deletions': pr_data['deletions'],
            'created_at': pr_data['created_at'],
            'closed_at': pr_data['closed_at'],
            'user': pr_data['user_data']['login']
        }
        if (pr_data['merged']):
            summary['merged_at'] = pr_data['merged_at']
        else:
            summary['merged_at'] = None
        return summary;
    
    def __init__(self, path):
        """
           Initilizes self.df, the dataframe with one row per pull_request.
        """

        self.df = pd.DataFrame(columns=['base_repo','title', 'state', 'commits', 'commits_data',
                                        'comments', 'changed_files', 'additions', 'deletions', 'created_at'
                                        'closed_at','user', 'merged_at'])
        pull_requests= []
        with open(path) as pr_file:
            for line in pr_file:
                line = json.loads(line)
                if(line['category']=="pull_request"):
                    pr = line
                    pull_requests.append(self._summary(pr_data=pr['data']))

        self.df = self.df.append(pull_requests, sort=False)
        #self.df['author_date'] = pd.to_datetime(self.df['author_date'], utc=True)
        #self.df['commit_date'] = pd.to_datetime(self.df['commit_date'], utc=True)
        
    def total_count(self):
        
        return len(self.df.index)
    
    def open_prs(self):
        return len(self.df.index[self.df['state']=='open'])
    
    def closed_prs(self):
        return len(self.df.index[self.df['state']=='closed'])
    
    def unique_users(self):
        return self.df.user.nunique()


In [133]:
pull_reqs = pr_statistics('./tf_analysis.json')
print('Number of open prs', pull_reqs.open_prs())
print('Number of closed prs', pull_reqs.closed_prs())
print('Number of unique users', pull_reqs.unique_users())

Number of open prs 60
Number of closed prs 117
Number of unique users 36


In [110]:
with open('./tf_analysis.json') as issue_file:
            for line in issue_file:
                line = json.loads(line)
                if(line['category']=="issue"):
                    issue = line
                    break
pprint(issue)

In [121]:
class issue_statistics:
    
    @staticmethod
    def _summary(issue_data):
        """Compute a summary of a pull_request, suitable as a row in a dataframe"""
        
        summary = {
            'title': issue_data['title'],
            'state': issue_data['state'],
            'comments':issue_data['comments'],
            'created_at': issue_data['created_at'],
            'closed_at': issue_data['closed_at'],
            'created_by': issue_data['user_data']['login']
        }
        return summary;
    
    def __init__(self, path):
        """
           Initilizes self.df, the dataframe with one row per issue.
        """

        self.df = pd.DataFrame(columns=['title', 'state', 'created_at', 'closed_at', 'comments', 'created_by'])
        issues = []
        with open(path) as issue_file:
            for line in issue_file:
                line = json.loads(line)
                if(line['category']=="issue"):
                    issue = line
                    issues.append(self._summary(issue_data=issue['data']))

        self.df = self.df.append(issues, sort=False)
        #self.df['author_date'] = pd.to_datetime(self.df['author_date'], utc=True)
        #self.df['commit_date'] = pd.to_datetime(self.df['commit_date'], utc=True)
        
    def total_count(self):        
        return len(self.df.index)
    
    def open_issues(self):
        return len(self.df.index[self.df['state']=='open'])
    
    def closed_issues(self):
        return len(self.df.index[self.df['state']=='closed'])
    
    def unique_users(self):
        return self.df.created_by.nunique()

In [122]:
issues = issue_statistics('./tf_analysis.json'
                         )
print('Number of open issues', issues.open_issues())
print('Number of closed issues', issues.closed_issues())
print('Number of unique issue creators', issues.unique_users())

Number of open issues 152
Number of closed issues 171
Number of unique issue creators 80


In [123]:
issues.df.head()

,title,state,created_at,closed_at,comments,created_by
0,Example in README not working,closed,2018-09-17T18:42:13Z,2018-09-17T21:56:55Z,2,piyush-kgp
1,Fixes bug - datasets.load was not implemented,closed,2018-09-17T19:57:28Z,2018-09-17T21:56:03Z,1,piyush-kgp
2,error on pip install,closed,2018-09-26T12:32:21Z,2018-10-01T07:16:31Z,1,tiaguinho
3,Import error on Windows,closed,2018-09-14T10:09:00Z,2018-10-01T07:17:00Z,3,LoSealL
4,[Question] - Would it be okay to generate tfre...,closed,2018-11-28T19:33:58Z,2018-11-29T19:11:58Z,1,ksachdeva
